In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pandas_profiling as pp

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, fbeta_score, roc_auc_score
from sklearn.metrics import classification_report as cr

In [4]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [5]:
df = pd.read_csv("C:\\Users\\CH40068799\\OneDrive - Wipro\\Documents\\Data Science\\ML projects files\\Project 6\\Property_train.csv")
df.head()

,Junk,InteriorsStyle,PriceIndex8,ListDate,Material,PriceIndex9,Agency,AreaIncomeType,EnvRating,PriceIndex7,...,Region,PriceIndex5,SubModel,Facade,State,NormalisedPopulation,BuildYear,RegionType,PropertyAge,PriceIndex2
0,0,6LLJ,14674.0,9/7/2010,UT7W,8270,CAT3,B,missing,13143.0,...,A,9022.0,6E9G,GREEN,MD,42077,2008,A,2,10692.0
1,1,XZ9F,4172.0,1/7/2009,PRN0,3890,CAT2,B,missing,3461.0,...,A,2742.0,FIVQ,SILVER,OK,77258,2002,A,7,3615.0
2,0,QWEV,8210.0,6/23/2010,A9RA,5900,CAT1,B,missing,6922.0,...,A,3768.0,4K0H,SILVER,SC,94514,2004,A,6,5805.0
3,0,MK2K,7309.0,3/19/2009,RRJQ,7460,OTHER,A,missing,6300.0,...,A,5370.0,ZLXY,WHITE,NC,82302,2002,C,7,6444.0
4,0,GK4G,9182.0,3/3/2010,HE5A,6400,OTHER,B,missing,8330.0,...,A,4803.0,FIVQ,SILVER,SC,49176,2007,B,3,6286.0


In [6]:
print(df.shape)

(62035, 31)


In [7]:
df.columns

Index(['Junk', 'InteriorsStyle', 'PriceIndex8', 'ListDate', 'Material',
       'PriceIndex9', 'Agency', 'AreaIncomeType', 'EnvRating', 'PriceIndex7',
       'ExpeditedListing', 'PriceIndex4', 'PriceIndex1', 'PriceIndex6',
       'PRIMEUNIT', 'Channel', 'Zip', 'InsurancePremiumIndex', 'PlotType',
       'Architecture', 'PriceIndex3', 'Region', 'PriceIndex5', 'SubModel',
       'Facade', 'State', 'NormalisedPopulation', 'BuildYear', 'RegionType',
       'PropertyAge', 'PriceIndex2'],
      dtype='object')

In [8]:
df.describe()

,Junk,PriceIndex9,ExpeditedListing,Zip,InsurancePremiumIndex,NormalisedPopulation,BuildYear,PropertyAge
count,62035.000000,62035.000000,62035.000000,62035.000000,62035.000000,62035.000000,62035.000000,62035.00000
mean,0.122544,6731.444765,0.024680,58038.149109,1277.286854,71462.694302,2005.347739,4.17171
std,0.327915,1766.130806,0.155148,26143.635828,600.787708,14583.079578,1.730014,1.70878
min,0.000000,225.000000,0.000000,2764.000000,462.000000,4825.000000,2001.000000,0.00000
25%,0.000000,5430.000000,0.000000,32124.000000,837.000000,61803.500000,2004.000000,3.00000
50%,0.000000,6705.000000,0.000000,73108.000000,1155.000000,73361.000000,2005.000000,4.00000
75%,0.000000,7900.000000,0.000000,80022.000000,1623.000000,82370.500000,2007.000000,5.00000
max,1.000000,38785.000000,1.000000,99224.000000,7498.000000,115717.000000,2010.000000,9.00000


In [9]:
list(zip(df.columns, df.dtypes, df.nunique()))

[('Junk', dtype('int64'), 2),
 ('InteriorsStyle', dtype('O'), 1036),
 ('PriceIndex8', dtype('O'), 12686),
 ('ListDate', dtype('O'), 516),
 ('Material', dtype('O'), 134),
 ('PriceIndex9', dtype('int64'), 2019),
 ('Agency', dtype('O'), 5),
 ('AreaIncomeType', dtype('O'), 4),
 ('EnvRating', dtype('O'), 3),
 ('PriceIndex7', dtype('O'), 12078),
 ('ExpeditedListing', dtype('int64'), 2),
 ('PriceIndex4', dtype('O'), 13006),
 ('PriceIndex1', dtype('O'), 10053),
 ('PriceIndex6', dtype('O'), 10863),
 ('PRIMEUNIT', dtype('O'), 3),
 ('Channel', dtype('O'), 3),
 ('Zip', dtype('int64'), 152),
 ('InsurancePremiumIndex', dtype('int64'), 277),
 ('PlotType', dtype('O'), 13),
 ('Architecture', dtype('O'), 33),
 ('PriceIndex3', dtype('O'), 12309),
 ('Region', dtype('O'), 3),
 ('PriceIndex5', dtype('O'), 10011),
 ('SubModel', dtype('O'), 831),
 ('Facade', dtype('O'), 17),
 ('State', dtype('O'), 37),
 ('NormalisedPopulation', dtype('int64'), 36675),
 ('BuildYear', dtype('int64'), 10),
 ('RegionType', dtype(

In [10]:
cutoff = (5/100)* df.shape[0]
cutoff

3101.75

In [11]:
numcol = ["PriceIndex5","PriceIndex7","PriceIndex4","PriceIndex1","PriceIndex6","PriceIndex2","PriceIndex3","PriceIndex8", "PriceIndex9"]
for col in numcol:
  df[col] = pd.to_numeric(df[col], errors="coerce")
list(zip(df.columns,df.dtypes,df.nunique()))

[('Junk', dtype('int64'), 2),
 ('InteriorsStyle', dtype('O'), 1036),
 ('PriceIndex8', dtype('float64'), 12685),
 ('ListDate', dtype('O'), 516),
 ('Material', dtype('O'), 134),
 ('PriceIndex9', dtype('int64'), 2019),
 ('Agency', dtype('O'), 5),
 ('AreaIncomeType', dtype('O'), 4),
 ('EnvRating', dtype('O'), 3),
 ('PriceIndex7', dtype('float64'), 12077),
 ('ExpeditedListing', dtype('int64'), 2),
 ('PriceIndex4', dtype('float64'), 13005),
 ('PriceIndex1', dtype('float64'), 10052),
 ('PriceIndex6', dtype('float64'), 10862),
 ('PRIMEUNIT', dtype('O'), 3),
 ('Channel', dtype('O'), 3),
 ('Zip', dtype('int64'), 152),
 ('InsurancePremiumIndex', dtype('int64'), 277),
 ('PlotType', dtype('O'), 13),
 ('Architecture', dtype('O'), 33),
 ('PriceIndex3', dtype('float64'), 12308),
 ('Region', dtype('O'), 3),
 ('PriceIndex5', dtype('float64'), 10010),
 ('SubModel', dtype('O'), 831),
 ('Facade', dtype('O'), 17),
 ('State', dtype('O'), 37),
 ('NormalisedPopulation', dtype('int64'), 36675),
 ('BuildYear', d

In [12]:
cols = df.select_dtypes(["object"])
for colname in cols:
    cats = df[colname].value_counts()
    keepcat = cats.index[cats >= cutoff]
    for cat in keepcat:
        if cat == "missing":
            continue
        name = colname+'_'+cat
        df[name] = (df[colname]==cat).astype(int)
    del df[colname]
    print(colname)
df.head()

InteriorsStyle
ListDate
Material
Agency
AreaIncomeType
EnvRating
PRIMEUNIT
Channel
PlotType
Architecture
Region
SubModel
Facade
State
RegionType


,Junk,PriceIndex8,PriceIndex9,PriceIndex7,ExpeditedListing,PriceIndex4,PriceIndex1,PriceIndex6,Zip,InsurancePremiumIndex,...,State_TX,State_FL,State_CA,State_NC,State_AZ,State_CO,State_SC,RegionType_A,RegionType_B,RegionType_C
0,0,14674.0,8270,13143.0,0,14224.0,9217.0,10387.0,21075,623,...,0,0,0,0,0,0,0,1,0,0
1,1,4172.0,3890,3461.0,0,4404.0,2958.0,3400.0,73129,1689,...,0,0,0,0,0,0,0,1,0,0
2,0,8210.0,5900,6922.0,0,8952.0,4568.0,4862.0,29697,2351,...,0,0,0,0,0,0,1,1,0,0
3,0,7309.0,7460,6300.0,0,7460.0,5361.0,6305.0,28273,1933,...,0,0,0,1,0,0,0,0,0,1
4,0,9182.0,6400,8330.0,0,9846.0,5428.0,5718.0,29697,482,...,0,0,0,0,0,0,1,0,1,0


In [13]:
print(df.shape)
df.dropna(axis=0, inplace=True)
print(df.shape)

(62035, 60)
(61763, 60)


In [14]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                   results['mean_test_score'][candidate],
                   results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            

In [15]:
def grid_search_wrapper(refit_score='recall_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization
    prints classifier performance metrics
    """
    skf = StratifiedKFold(n_splits=10)
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,
                              cv=skf, return_train_score=True, n_jobs=-1, verbose=30)
    grid_search.fit(xtrain.values, ytrain.values)
    
    #make predictions
    ypred = grid_search.predict(xtest.values)
    
    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)
    # confusion matrix on the test data
    print('\nConfusion matrix of Random forest optimized for {} on the test data:'.format(refit_score))
    print(pd.DataFrame(confusion_matrix(ytest, ypred),
                      columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))
    return grid_search

In [16]:
xtrain, xtest, ytrain, ytest = train_test_split(df[df.columns[1:]], df.Junk, test_size=0.3, stratify=df.Junk)

In [17]:
#deciding Beta value for model :
clf = RandomForestClassifier()
param_grid = {
    "class_weight":[{1:0.122,0:0.877}],
    "n_estimators":[500],
   # 'min_samples_split': [3, 5, 10], 
    #'max_depth': [10, 20, 30],
   # 'max_features': [3, 5, 10, 20]
}



In [ ]:
for beta_val in {0.5,0.7,1,1.5,2,5,10,20,40,70,90}:
  scorers = {
      'precision_score': make_scorer(precision_score),
      'recall_score': make_scorer(recall_score),
      'accuracy_score': make_scorer(accuracy_score),
      'fbeta_score': make_scorer(fbeta_score, beta=beta_val)
  }
  print(beta_val)
  grid_search_clf = grid_search_wrapper(refit_score='fbeta_score')

0.5
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16145       112
pos      1805       467
0.7
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16131       126
pos      1809       463
2
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best params for fbeta_score
{'class_weight': {1: 0.122, 0: 0.877}, 'n_estimators': 500}

Confusion matrix of Random forest optimized for fbeta_score on the test data:
     pred_neg  pred_pos
neg     16140       117
pos      1811       461
1
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best params for fbeta_score

In [ ]:
for beta_val in {200,500,1000,1500,2000}:
    scorers = {
      'precision_score': make_scorer(precision_score),
      'recall_score': make_scorer(recall_score),
      'accuracy_score': make_scorer(accuracy_score),
      'fbeta_score': make_scorer(fbeta_score, beta=beta_val)
    }
    print(beta_val)
    grid_search_clf = grid_search_wrapper(refit_score='fbeta_score')

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()
param_grid = {
    'learning_rate':[0.9],
    'class_weight':[{1: 0.122, 0: 0.877}],
    'n_estimators':[500]
}



In [ ]:
for beta_val in {0,1,100,200,500,1000}:
    scorers ={
    'precision_score': make_scorer(precision_score),
      'recall_score': make_scorer(recall_score),
      'accuracy_score': make_scorer(accuracy_score),
      'fbeta_score': make_scorer(fbeta_score, beta=beta_val)
    }
    print(beta_val)
    grid_search_clf = grid_search_wrapper(refit_score='fbeta_score')

In [ ]:
scorers = {
     'precision_score': make_scorer(precision_score),
     'recall_score': make_scorer(recall_score),
     'accuracy_score': make_scorer(accuracy_score),
     'fbeta_score': make_scorer(fbeta_score, beta=200),
     'roc_auc_score':make_scorer(roc_auc_score)
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier()
param_grid = {
    "class_weight":[{1:0.122,0:0.877}],
    "n_estimators":[500]
}
grid_search_clf = grid_search_wrapper(refit_score='precision_score')

In [ ]:
clf = RandomForestClassifier()
param_grid = {
    "class_weight":[{1:0.122,0:0.877}],
    "n_estimators":[500]
}
grid_search_clf = grid_search_wrapper(refit_score='recall_score')

In [ ]:
clf = RandomForestClassifier()
param_grid = {
    "class_weight":[{1:0.122,0:0.877}],
    "n_estimators":[500]
}
grid_search_clf = grid_search_wrapper(refit_score='fbeta_score')

In [ ]:
!pip install rfpimp
from rfpimp import *

In [ ]:
# adding a column in X with random values to help find out correct feature importance and features to be removed
xtrain['random']=np.random.choice(range(xtrain.shape[0]), xtrain.shape[0])
xtest['random']=np.random.choice(range(xtrain.shape[0]), xtest.shape[0])

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()
rf=XGBClassifier(base_score=0.5, booster='gbtree',
                class_weight={0:0.877, 1:0.122}, colsample_bylevel=1,
                colsample_bynode=1, colsample_bytree=1, gamma=0,
                learning_rate=0.9, max_delta_step=0, max_depth=3,
                min_child_weight=1, missing=1, n_estimators=500, njobs=-1,
                nthread=None, objective='binary:logistic', random_state=0,
                reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                silent=None, subsample=1, verbosity=1)

In [ ]:
xtest.shape

In [ ]:
rf.fit(xtrain, ytrain)

In [ ]:
list(zip(df.columns, rf.feature_importances_))
imp = importances(rf, xtrain, ytrain, n_samples=-1)
viz = plot_importances(imp)
viz.view()

In [ ]:
imp

In [ ]:
indx = imp[imp.values > 0.018943].index.values
xtrain[indx]

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [0.01, 0.05,0.1,0.25, 0.5, 0.7],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500]
        #'max_depth': [3, 5, 7, 10],
        #'min_child_weight': [1, 3, 5],
        #'subsample': [0.5, 0.7],
        #'colsample_bytree': [0.5, 0.7]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.7,0.9,0.8],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [3, 5, 7, 10,15]
        #'min_child_weight': [1, 3, 5],
        #'subsample': [0.5, 0.7],
        #'colsample_bytree': [0.5, 0.7]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)


In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [1, 3, 5,10,15,50,100,75,125]
        #'subsample': [0.5, 0.7],
        'colsample_by_tree': [0.5, 0.7]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [10],
        'subsample': [0.2,0.3,0.5,0.6, 0.7,0.9],
        #'colsample_bytree': [0.5, 0.7]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [10],
        'subsample': [0.3],
        'scale_pos_weight': [0.2,0.3,0.5,0.6, 0.7,0.9]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [10],
        'subsample': [0.3],
        'scale_pos_weight': [0.2,0.3,0.5,0.6, 0.7,0.9]
}

grid_search_clf = grid_search_wrapper(refit_score='recall_score')

gsf= grid_search_clf.best_estimator_
gsf.fit(xtrain[indx],ytrain)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier()

param_grid = {
        'learning_rate': [ 0.9],
        'class_weight': [{1: 0.122, 0: 0.877}], 'n_estimators': [500],
        'max_depth': [5],
        'min_child_weight': [10],
        'subsample': [0.3],
        'scale_pos_weight': [0.8,0.9,1,9]
}

grid_search_clf = grid_search_wrapper(refit_score='roc_auc_score')

gsf= grid_search_clf.best_estimator_



In [ ]:
gsf=XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight={0: 0.877, 1: 0.122}, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.9, max_delta_step=0, max_depth=5,
              min_child_weight=10, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=0.9, seed=None,
              silent=None, subsample=0.3, verbosity=1)

In [ ]:
gsf.fit(xtrain[indx],ytrain)
preds = gsf.predict(xtest[indx])
roc_auc_score(ytest, preds)

In [ ]:
from xgboost import XGBClassifier
xgb_model= XGBClassifier()
param_tuning = {
        'learning_rate': [0.25],
        "class_weight":[{0:0.877,1:0.123}],
        'max_depth': [3],
        'min_child_weight': [5],
        'subsample': [0.7],
        'colsample_bytree': [0.05,0.1,0.2,0.5, 0.7,0.9],
        'n_estimators' : [500] 
}

xgbsearch3 = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           cv = 5,scoring ='roc_auc',
                           n_jobs = 20,
                           verbose = 30)

xgbsearch3.fit(xtrain[indx],ytrain)
print(xgbsearch3.best_params_)
report(xgbsearch3.cv_results_,5)

In [ ]:
xgbfinal = xgbsearch3.best_estimator_
xgbfinal.fit(xtrain[indx], ytrain)
preds = xgbfinal.predict(xtest[indx])
print(confusion_matrix(ytest, preds))